# Hardware Requirements

The overall training process takes around **5 hours** using a single P100 GPU (tested on Kaggle). <br>
On Google Colab with a T4 GPU, it will take longer, roughly **8 hours** (Not recommended since Colab might automatically lose connection)

GPU RAM usage:  14GB <br>
Disk: 10 GB

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Install Dependencies

In [2]:
!pip install numpy tqdm nltk transformers==4.31.0 peft==0.4.0 accelerate==0.21.0 bitsandbytes==0.41.1 pandas

# Download the PDF-VQA Dataset

**Note:** Although in the provided [**strong baseline juypter notebook**](https://colab.research.google.com/drive/1C4HgX_Yl3vc6u5Wq6duZe0YQBU34ooh2?usp=sharing) there is a way of downloading the files via gdrive API, we noticed that in this way the downloaded validation file "val_dataframe.csv" **DOES NOT** contain the ground-truth answer, which make it impossible to use the validation set for validating the training.
Therefore, we have to download the full dataset from kaggle instead.

Download the full dataset from the Kaggle competition data page: https://www.kaggle.com/competitions/pdfvqa/data, by clicking the button "Download All" in the bottom right. Unzip, and put the downloaded folder "**pdfvqa**" to the same folder as this notebook.

Alternatively, you can try the following command: (This might not work after certain time, since the downloading link was dynamically allocated.)

In [3]:
!curl -o pdfvqa.zip 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/56858/6336428/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1696621416&Signature=MD2fz4I6wSiWcaViYmZ31JBfS5A4Qlcsjt557vgdgLydNC5uCRwH21OcX9AzJ4AbYEd9gt9w%2FACLBd%2BUEuMd8%2FLaJ5unJWcgSbERUDLnLCr9p1SE1UCwseAbM2FnocPC3yelMTTjZ8SoMq49rZuB3Fx%2BbCqS6ddTmssDhPZQ5tOrdQD31PDOBXxbmoZxx7JQco6%2FOCMXy%2BoWaeDf9leZfOro5YfNu%2FANo%2FoUDzLQ2kPxL%2B%2F2oGMFpFmoM60zaupqBsaUJz0XIKnO0LfiT7kv6Jd%2BOJLD45iRyXkcHIUYuTCNvTKSjABqelJjDloVRPLZfV1%2BrCNc2C%2BtzaeJSH2sdg%3D%3D&response-content-disposition=attachment%3B+filename%3Dpdfvqa.zip'
!unzip -q pdfvqa.zip -d pdfvqa/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2103M  100 2103M    0     0  26.1M      0  0:01:20  0:01:20 --:--:-- 27.3M:19  0:01:02 28.8M20  0:00:37  0:00:43 27.9M0  0:01:16  0:00:04 29.9M


Or, alternatively, use the kaggle API: ([**User authentication of Kaggle API is needed**](https://www.kaggle.com/discussions/general/74235))

In [4]:
# !kaggle competitions download -c pdfvqa

### Here is the structure of the data folder
```
pdfvqa
├── test_dataframe_without_answer.csv
├── test_doc_info.pkl
├── test_images
├── train_dataframe.csv
├── train_doc_info.pkl
├── train_images
├── val_dataframe.csv
├── val_doc_info.pkl
└── val_images
```

# Code for the Model

## Utils

In [5]:
import os
from glob import glob
import torch
import torch.nn as nn
import json
from tqdm import tqdm

def load_model( model_folder ):
    ckpt_list =  glob( model_folder + "/*.pt" )
    if len( ckpt_list ) >0:
        ckpt_list.sort( key = os.path.getmtime )
        ckpt_name = ckpt_list[-1]
        if torch.cuda.is_available():
            ckpt = torch.load( ckpt_name )
        else:
            ckpt = torch.load( ckpt_name,  map_location=torch.device('cpu') )
    else:
        ckpt = None
    return ckpt

def save_model(  module_dicts ,save_name , max_to_keep = 0, overwrite = True ):
    folder_path = os.path.dirname( os.path.abspath( save_name )  )
    if not os.path.exists( folder_path  ):
        os.makedirs( folder_path )

    state_dicts = {}
    for key in module_dicts.keys():
        if isinstance( module_dicts[key], nn.DataParallel ):
            state_dicts[key] = module_dicts[key].module.state_dict()
        elif isinstance( module_dicts[key], nn.Module ):
            state_dicts[key] = module_dicts[key].state_dict()
        else:
            state_dicts[key] = module_dicts[key]

    if os.path.exists( save_name ):
        if overwrite:
            os.remove( save_name )
            torch.save( state_dicts, save_name )
        else:
            print("Warning: checkpoint file already exists!")
            return
    else:
        torch.save( state_dicts, save_name )

    if max_to_keep > 0:
        pt_file_list = glob(folder_path+"/*.pt")
        pt_file_list.sort( key= lambda x: os.path.getmtime(x) )
        for idx in range( len( pt_file_list ) - max_to_keep ):
            os.remove( pt_file_list[idx]  )


def get_lr(optimizer):
    return [param_group['lr'] for param_group in optimizer.param_groups ]

## Datautils

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from copy import deepcopy
from tqdm import tqdm

import ast, pickle
import pandas as pd

from nltk.tokenize import RegexpTokenizer
word_tok = RegexpTokenizer(r"\w+")


## the corpus has been preprocessed, so here only lower is needed
## all digits are kept, since sent2vec unigram embedding has digit embedding
## no stemming, no lemmatization
class SentenceTokenizer:
    def __init__(self ):
        pass
    def tokenize(self, sen ):
        return sen.lower()

class Vocab:
    def __init__(self, words, eos_token = "<eos>", pad_token = "<pad>", unk_token = "<unk>" ):
        self.words = words
        self.index_to_word = {}
        self.word_to_index = {}
        for idx in range( len(words) ):
            self.index_to_word[ idx ] = words[idx]
            self.word_to_index[ words[idx] ] = idx
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.eos_index = self.word_to_index[self.eos_token]
        self.pad_index = self.word_to_index[self.pad_token]

        self.tokenizer = SentenceTokenizer()

    def index2word( self, idx ):
        return self.index_to_word.get( idx, self.unk_token)
    def word2index( self, word ):
        return self.word_to_index.get( word, -1 )
    # The sentence needs to be tokenized
    def sent2seq( self, sent, max_len = None , tokenize = True):
        if tokenize:
            sent = self.tokenizer.tokenize(sent)
        seq = []
        for w in sent.split():
            if w in self.word_to_index:
                seq.append( self.word2index(w) )
        if max_len is not None:
            if len(seq) >= max_len:
                seq = seq[:max_len -1]
                seq.append( self.eos_index )
            else:
                seq.append( self.eos_index )
                seq += [ self.pad_index ] * ( max_len - len(seq) )
        return seq
    def seq2sent( self, seq ):
        sent = []
        for i in seq:
            if i == self.eos_index or i == self.pad_index:
                break
            sent.append( self.index2word(i) )
        return " ".join(sent)

class ExtractionDataset(Dataset):
    def __init__( self,  corpus, vocab , max_seq_len , max_doc_len  ):
        self.vocab = vocab
        self.max_seq_len = max_seq_len
        self.max_doc_len = max_doc_len
        ## corpus is a list
        self.corpus = corpus

    def __len__(self):
        return len(self.corpus)

    def __getitem__( self, idx ):

        doc_data = self.corpus[idx]
        sentences = doc_data["text"]
        valid_sen_idxs = deepcopy( doc_data["indices"] )

        sentences = sentences[:self.max_doc_len]
        num_sentences_in_doc = len( sentences )

        np.random.shuffle( valid_sen_idxs )
        valid_sen_idxs = np.array( valid_sen_idxs )
        valid_sen_idxs = valid_sen_idxs[ valid_sen_idxs < num_sentences_in_doc ]
        valid_sen_idxs = valid_sen_idxs[:self.max_doc_len]

        stop_loss_mask = np.zeros( self.max_doc_len ).astype(np.float32)
        stop_loss_mask[:len(valid_sen_idxs) + 1] = 1.0

        valid_sen_idxs = np.array(valid_sen_idxs.tolist() + [-1] * ( self.max_doc_len - len(valid_sen_idxs)))

        sentences += [""] * ( self.max_doc_len - num_sentences_in_doc )
        doc_mask = np.array(  [ True if sen.strip() == "" else False for sen in  sentences   ]  )

        seqs = [  self.vocab.sent2seq( sen, self.max_seq_len ) for sen in sentences ]
        seqs = np.asarray( seqs )

        return seqs, valid_sen_idxs, doc_mask, stop_loss_mask

def tokenize_sent(sent):
    return " ".join( word_tok.tokenize( sent ) )


def load_dataset_for_inference( doc_info_path, dataframe_path ):
    doc_info = pickle.load(open(doc_info_path, "rb"))
    paper_dict = {}
    for count, pmcid in enumerate(doc_info):
        paper = doc_info[pmcid]
        paper_info = []
        for _, page in paper["pages"].items():
            for obj_id in page.get("ordered_id", []):
                paper_info.append( {
                    "global_id":page["objects"][obj_id]["global_id"],
                    "text":page["objects"][obj_id].get("text", "").replace("\n", " ")
                } )
        paper_dict[pmcid] = paper_info

    query_data = pd.read_csv(dataframe_path)
    if "global_id" in query_data:
        query_data["global_id"] = query_data["global_id"].apply(ast.literal_eval)

    corpus = []
    for pos in range(len( query_data )):
        question = query_data["question"][pos]
        question_type = query_data["question_type"][pos]
        pmcid = str(query_data["pmcid"][pos])
        if "global_id" in query_data:
            answer_global_ids = set(query_data["global_id"][pos])
        else:
            answer_global_ids = None

        corpus.append( {
            "question":question,
            "question_type":question_type,
            "pmcid":pmcid,
            "answer_global_ids":answer_global_ids
        } )

    return corpus, paper_dict

def load_corpus( doc_info_path, dataframe_path  ):

    doc_info = pickle.load(open(doc_info_path, "rb"))
    query_data = pd.read_csv(dataframe_path)
    query_data["global_id"] = query_data["global_id"].apply(ast.literal_eval)

    paper_dict = {}
    for count, pmcid in enumerate(doc_info):
        paper = doc_info[pmcid]
        paper_info = []
        for _, page in paper["pages"].items():
            for obj_id in page.get("ordered_id", []):
                paper_info.append( {
                    "global_id":page["objects"][obj_id]["global_id"],
                    "text":page["objects"][obj_id].get("text", "").replace("\n", " ")
                } )
        paper_dict[pmcid] = paper_info

    corpus = []
    for pos in range(len( query_data )):
        question = query_data["question"][pos]
        question_type = query_data["question_type"][pos]
        pmcid = str(query_data["pmcid"][pos])
        answer_global_ids = set(query_data["global_id"][pos])

        # valid_indices = []
        # sentence_list = [ question_type.replace("_", " ") + " : " + question ]
        # for text_block in paper_dict[pmcid]:
        #     if text_block["global_id"] in answer_global_ids:
        #         valid_indices.append( len( sentence_list ) )
        #     sentence_list.append( text_block["text"] )

        valid_indices = []
        sentence_list = [] #[ question + " : " + question_type.replace("_", " ") ]
        for text_block in paper_dict[pmcid]:
            if text_block["global_id"] in answer_global_ids:
                valid_indices.append( len( sentence_list ) )
            sentence_list.append(  question + " " + question_type.replace("_", " ") + " " + text_block["text"] )

        data_example = {
            "text":[ tokenize_sent(sen) for sen in sentence_list],
            "indices":valid_indices
        }

        corpus.append( data_example )
    return corpus

## Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from torch.distributions import Categorical

class AddMask( nn.Module ):
    def __init__( self, pad_index ):
        super().__init__()
        self.pad_index = pad_index
    def forward( self, x):
        # here x is a batch of input sequences (not embeddings) with the shape of [ batch_size, seq_len]
        mask = x == self.pad_index
        return mask

class PositionalEncoding( nn.Module ):
    def __init__(self,  embed_dim, max_seq_len = 512  ):
        super().__init__()
        self.embed_dim = embed_dim
        self.max_seq_len = max_seq_len
        pe = torch.zeros( 1, max_seq_len,  embed_dim )
        for pos in range( max_seq_len ):
            for i in range( 0, embed_dim, 2 ):
                pe[ 0, pos, i ] = math.sin( pos / ( 10000 ** ( i/embed_dim ) )  )
                if i+1 < embed_dim:
                    pe[ 0, pos, i+1 ] = math.cos( pos / ( 10000** ( i/embed_dim ) ) )
        self.register_buffer( "pe", pe )
        ## register_buffer can register some variables that can be saved and loaded by state_dict, but not trainable since not accessible by model.parameters()
    def forward( self, x ):
        return x + self.pe[ :, : x.size(1), :]

class MultiHeadAttention( nn.Module ):
    def __init__(self, embed_dim, num_heads ):
        super().__init__()
        dim_per_head = int( embed_dim/num_heads )

        self.ln_q = nn.Linear( embed_dim, num_heads * dim_per_head )
        self.ln_k = nn.Linear( embed_dim, num_heads * dim_per_head )
        self.ln_v = nn.Linear( embed_dim, num_heads * dim_per_head )

        self.ln_out = nn.Linear( num_heads * dim_per_head, embed_dim )

        self.num_heads = num_heads
        self.dim_per_head = dim_per_head

    def forward( self, q,k,v, mask = None):
        q = self.ln_q( q )
        k = self.ln_k( k )
        v = self.ln_v( v )

        q = q.view( q.size(0), q.size(1),  self.num_heads, self.dim_per_head  ).transpose( 1,2 )
        k = k.view( k.size(0), k.size(1),  self.num_heads, self.dim_per_head  ).transpose( 1,2 )
        v = v.view( v.size(0), v.size(1),  self.num_heads, self.dim_per_head  ).transpose( 1,2 )

        a = self.scaled_dot_product_attention( q,k, mask )
        new_v = a.matmul(v)
        new_v = new_v.transpose( 1,2 ).contiguous()
        new_v = new_v.view( new_v.size(0), new_v.size(1), -1 )
        new_v = self.ln_out(new_v)
        return new_v

    def scaled_dot_product_attention( self, q, k, mask = None ):
        ## note the here q and k have converted into multi-head mode
        ## q's shape is [ Batchsize, num_heads, seq_len_q, dim_per_head ]
        ## k's shape is [ Batchsize, num_heads, seq_len_k, dim_per_head ]
        # scaled dot product
        a = q.matmul( k.transpose( 2,3 ) )/ math.sqrt( q.size(-1) )
        # apply mask (either padding mask or seqeunce mask)
        if mask is not None:
            a = a.masked_fill( mask.unsqueeze(1).unsqueeze(1) , -1e9 )
        # apply softmax, to get the likelihood as attention matrix
        a = F.softmax( a, dim=-1 )
        return a

class FeedForward( nn.Module ):
    def __init__( self, embed_dim, hidden_dim ):
        super().__init__()
        self.ln1 = nn.Linear( embed_dim, hidden_dim )
        self.ln2 = nn.Linear( hidden_dim, embed_dim )
    def forward(  self, x):
        net = F.relu(self.ln1(x))
        out = self.ln2(net)
        return out

class TransformerDecoderLayer( nn.Module ):
    def __init__(self, embed_dim, num_heads, hidden_dim ):
        super().__init__()
        self.masked_mha = MultiHeadAttention(  embed_dim, num_heads )
        self.norm1 = nn.LayerNorm( embed_dim )
        self.mha = MultiHeadAttention( embed_dim, num_heads )
        self.norm2 = nn.LayerNorm( embed_dim )
        self.feed_forward = FeedForward( embed_dim, hidden_dim )
        self.norm3 = nn.LayerNorm( embed_dim )
    def forward(self, encoder_output, x, src_mask, trg_mask , dropout_rate = 0. ):
        short_cut = x
        net = F.dropout(self.masked_mha( x,x,x, trg_mask ), p = dropout_rate)
        net = self.norm1( short_cut + net )
        short_cut = net
        net = F.dropout(self.mha( net, encoder_output, encoder_output, src_mask ), p = dropout_rate)
        net = self.norm2( short_cut + net )
        short_cut = net
        net = F.dropout(self.feed_forward( net ), p = dropout_rate)
        net = self.norm3( short_cut + net )
        return net

class MultiHeadPoolingLayer( nn.Module ):
    def __init__( self, embed_dim, num_heads  ):
        super().__init__()
        self.num_heads = num_heads
        self.dim_per_head = int( embed_dim/num_heads )
        self.ln_attention_score = nn.Linear( embed_dim, num_heads )
        self.ln_value = nn.Linear( embed_dim,  num_heads * self.dim_per_head )
        self.ln_out = nn.Linear( num_heads * self.dim_per_head , embed_dim )
    def forward(self, input_embedding , mask=None):
        a = self.ln_attention_score( input_embedding )
        v = self.ln_value( input_embedding )

        a = a.view( a.size(0), a.size(1), self.num_heads, 1 ).transpose(1,2)
        v = v.view( v.size(0), v.size(1),  self.num_heads, self.dim_per_head  ).transpose(1,2)
        a = a.transpose(2,3)
        if mask is not None:
            a = a.masked_fill( mask.unsqueeze(1).unsqueeze(1) , -1e9 )
        a = F.softmax(a , dim = -1 )

        new_v = a.matmul(v)
        new_v = new_v.transpose( 1,2 ).contiguous()
        new_v = new_v.view( new_v.size(0), new_v.size(1) ,-1 ).squeeze(1)
        new_v = self.ln_out( new_v )
        return new_v

class LocalSentenceEncoder( nn.Module ):
    def __init__( self, vocab_size, pad_index, embed_dim, num_heads , hidden_dim, pretrained_word_embedding ):
        super().__init__()
        self.addmask = AddMask( pad_index )

        self.rnn = nn.LSTM(  embed_dim, embed_dim, 2, batch_first = True, bidirectional = True)
        self.mh_pool = MultiHeadPoolingLayer( 2*embed_dim, num_heads )
        self.norm_out = nn.LayerNorm( 2*embed_dim )
        self.ln_out = nn.Linear( 2*embed_dim, embed_dim )

        if pretrained_word_embedding is not None:
            ## make sure the pad embedding is 0
            pretrained_word_embedding[pad_index] = 0
            self.register_buffer( "word_embedding", torch.from_numpy( pretrained_word_embedding ) )
        else:
            self.register_buffer( "word_embedding", torch.randn( vocab_size, embed_dim ) )

    """
    input_seq 's shape:  batch_size x seq_len
    """
    def forward( self, input_seq, dropout_rate = 0. ):
        mask = self.addmask( input_seq )
        ## batch_size x seq_len x embed_dim
        net = self.word_embedding[ input_seq ]
        net, _ = self.rnn( net )
        net =  self.ln_out(F.relu(self.norm_out(self.mh_pool( net, mask ))))
        return net

class GlobalContextEncoder(nn.Module):
    def __init__(self, embed_dim,  num_heads, hidden_dim ):
        super().__init__()
        self.rnn = nn.LSTM(  embed_dim, embed_dim, 2, batch_first = True, bidirectional = True)
        self.norm_out = nn.LayerNorm( 2*embed_dim )
        self.ln_out = nn.Linear( 2*embed_dim, embed_dim )

    def forward(self, sen_embed, doc_mask, dropout_rate = 0.):
        net, _ = self.rnn( sen_embed )
        net = self.ln_out(F.relu( self.norm_out(net) ) )
        return net

class ExtractionContextDecoder( nn.Module ):
    def __init__( self, embed_dim,  num_heads, hidden_dim, num_dec_layers ):
        super().__init__()
        self.pos_encode = PositionalEncoding( embed_dim)
        self.layer_list = nn.ModuleList( [  TransformerDecoderLayer( embed_dim, num_heads, hidden_dim ) for _ in range(num_dec_layers) ] )
    ## remaining_mask: set all unextracted sen indices as True
    ## extraction_mask: set all extracted sen indices as True
    def forward( self, sen_embed, remaining_mask, extraction_mask, dropout_rate = 0. ):
        # sen_embed = self.pos_encode( sen_embed )

        net = sen_embed
        for layer in self.layer_list:
            #  encoder_output, x,  src_mask, trg_mask , dropout_rate = 0.
            net = layer( sen_embed, net, remaining_mask, extraction_mask, dropout_rate )
        return net

class Extractor( nn.Module ):
    def __init__( self, embed_dim, num_heads ):
        super().__init__()
        self.norm_input = nn.LayerNorm( 3*embed_dim  )

        self.ln_hidden1 = nn.Linear(  3*embed_dim, 2*embed_dim  )
        self.norm_hidden1 = nn.LayerNorm( 2*embed_dim  )

        self.ln_hidden2 = nn.Linear(  2*embed_dim, embed_dim  )
        self.norm_hidden2 = nn.LayerNorm( embed_dim  )

        self.ln_out = nn.Linear(  embed_dim, 1 )

        self.mh_pool = MultiHeadPoolingLayer( embed_dim, num_heads )
        self.norm_pool = nn.LayerNorm( embed_dim  )
        self.ln_stop = nn.Linear(  embed_dim, 1 )

    def forward( self, sen_embed, relevance_embed, redundancy_embed , extraction_mask, dropout_rate = 0. ):
        if redundancy_embed is None:
            redundancy_embed = torch.zeros_like( sen_embed )
        net = self.norm_input( F.dropout( torch.cat( [ sen_embed, relevance_embed, redundancy_embed ], dim = 2 ) , p = dropout_rate  )  )
        net = F.relu( self.norm_hidden1( F.dropout( self.ln_hidden1( net ) , p = dropout_rate  )   ))
        hidden_net = F.relu( self.norm_hidden2( F.dropout( self.ln_hidden2( net)  , p = dropout_rate  )  ))

        sen_logits = self.ln_out( hidden_net ).squeeze(2)

        net = F.relu( self.norm_pool(  F.dropout( self.mh_pool( hidden_net, extraction_mask) , p = dropout_rate  )  ))
        stop_logits = self.ln_stop( net ).squeeze(1)

        return sen_logits, stop_logits

class MemSumPDFVQA:
    def __init__( self, model_path, vocabulary_path, gpu = None , embed_dim=200, num_heads=8, hidden_dim = 1024, num_ehe_layers = 3,  max_seq_len =200, max_doc_len = 200  ):
        with open( vocabulary_path , "rb" ) as f:
            words = pickle.load(f)
        self.vocab = Vocab( words )
        vocab_size = len(words)
        self.local_sentence_encoder = LocalSentenceEncoder( vocab_size, self.vocab.pad_index, embed_dim,num_heads,hidden_dim,None )
        self.global_context_encoder = GlobalContextEncoder( embed_dim, num_heads, hidden_dim )
        self.extraction_context_decoder = ExtractionContextDecoder( embed_dim, num_heads, hidden_dim, num_ehe_layers )
        self.extractor = Extractor( embed_dim, num_heads )
        ckpt = torch.load( model_path, map_location = "cpu" )
        self.local_sentence_encoder.load_state_dict( ckpt["local_sentence_encoder"] )
        self.global_context_encoder.load_state_dict( ckpt["global_context_encoder"] )
        self.extraction_context_decoder.load_state_dict( ckpt["extraction_context_decoder"] )
        self.extractor.load_state_dict(ckpt["extractor"])

        self.device =  torch.device( "cuda:%d"%(gpu) if gpu is not None and torch.cuda.is_available() else "cpu"  )
        self.local_sentence_encoder.to(self.device)
        self.global_context_encoder.to(self.device)
        self.extraction_context_decoder.to(self.device)
        self.extractor.to(self.device)

        self.sentence_tokenizer = SentenceTokenizer()
        self.max_seq_len = max_seq_len
        self.max_doc_len = max_doc_len

        self.word_tok = RegexpTokenizer(r"\w+")

    def tokenize_sent(self, sent):
        return " ".join( self.word_tok.tokenize( sent ) )


    def inference( self, question, question_type, doc_content, p_stop_thres = 0.2, max_extracted_sentences_per_document = 4 ):
        """doc_content is a list of { "global_id":global_id, "text":text }"""

        ## convert to sequence
        seqs = []
        doc_mask = []

        document = []
        for text_block in doc_content:
            document.append( self.tokenize_sent( question + " " + question_type.replace("_", " ") + " " + text_block["text"] ) )
        document = document[:self.max_doc_len]
        document = document + [""] * ( self.max_doc_len -  len(document) )

        doc_mask.append(  [ 1 if sen.strip() == "" else 0 for sen in  document   ] )
        document_sequences = []
        for sen in document:
            seq = self.vocab.sent2seq( sen, self.max_seq_len )
            document_sequences.append(seq)
        seqs.append(document_sequences)
        seqs = np.asarray(seqs)
        doc_mask = np.asarray(doc_mask) == 1
        seqs = torch.from_numpy(seqs).to(self.device)
        doc_mask = torch.from_numpy(doc_mask).to(self.device)

        num_documents = seqs.size(0)
        with torch.no_grad():
            num_sentences = seqs.size(1)
            sen_embed  = self.local_sentence_encoder( seqs.view(-1, seqs.size(2) )  )
            sen_embed = sen_embed.view( -1, num_sentences, sen_embed.size(1) )
            relevance_embed = self.global_context_encoder( sen_embed, doc_mask  )

            num_documents = seqs.size(0)
            doc_mask = doc_mask.detach().cpu().numpy()
            seqs = seqs.detach().cpu().numpy()

            extracted_sentences_positions = []

            for doc_i in range(num_documents):
                current_doc_mask = doc_mask[doc_i:doc_i+1]
                current_remaining_mask_np = np.ones_like(current_doc_mask ).astype(bool) | current_doc_mask
                current_extraction_mask_np = np.zeros_like(current_doc_mask).astype(bool) | current_doc_mask

                current_sen_embed = sen_embed[doc_i:doc_i+1]
                current_relevance_embed = relevance_embed[ doc_i:doc_i+1 ]
                current_redundancy_embed = None

                current_hyps = []

                for step in range( max_extracted_sentences_per_document+1 ) :
                    current_extraction_mask = torch.from_numpy( current_extraction_mask_np ).to(self.device)
                    current_remaining_mask = torch.from_numpy( current_remaining_mask_np ).to(self.device)
                    if step > 0:
                        current_redundancy_embed = self.extraction_context_decoder( current_sen_embed, current_remaining_mask, current_extraction_mask  )
                    sen_logits, stop_logits = self.extractor( current_sen_embed, current_relevance_embed, current_redundancy_embed , current_extraction_mask  )
                    sen_logits = sen_logits.masked_fill( current_extraction_mask, -1e9 )
                    p_stop = stop_logits.sigmoid()

                    stop = p_stop.item() > p_stop_thres
                    sen_i = sen_logits.argmax(dim=1)[0].item()

                    if stop or step == max_extracted_sentences_per_document or sen_i >= len(doc_content):
                        extracted_sentences_positions.append( current_hyps )
                        break
                    else:
                        current_hyps.append(sen_i)
                        current_extraction_mask_np[0, sen_i] = True
                        current_remaining_mask_np[0, sen_i] = False

            pred_indices = extracted_sentences_positions[0]
            if len(pred_indices) == 0:
                return [ -1 ]
            else:
                return [ doc_content[idx]["global_id"] for idx in pred_indices ]

# Training

## Training Preparation

In [8]:
import os
import pickle
from tqdm import tqdm
import time
import os, sys
import copy
import re
from huggingface_hub import snapshot_download
from transformers import set_seed
## download the pretrained glove word embedding (200 dimension)
snapshot_download('nianlong/memsum-word-embedding', local_dir = "model/word_embedding" )

def update_moving_average( m_ema, m, decay ):
    with torch.no_grad():
        param_dict_m_ema =  m_ema.module.parameters()  if isinstance(  m_ema, nn.DataParallel ) else m_ema.parameters() 
        param_dict_m =  m.module.parameters()  if isinstance( m , nn.DataParallel ) else  m.parameters() 
        for param_m_ema, param_m in zip( param_dict_m_ema, param_dict_m ):
            param_m_ema.copy_( decay * param_m_ema + (1-decay) *  param_m )
            

def LOG( info, log_out_file, end="\n" ):
    with open( log_out_file, "a" ) as f:
        f.write( info + end )

def train_iteration(batch):
    seqs, valid_sen_idxs, doc_mask, stop_loss_mask = [_.to(device) for _ in batch]
    num_documents = seqs.size(0)
    num_sentences = seqs.size(1)

    local_sen_embed = local_sentence_encoder( seqs.view(-1, seqs.size(2) ), dropout_rate )
    local_sen_embed = local_sen_embed.view( -1, num_sentences, local_sen_embed.size(1) )
    global_context_embed = global_context_encoder( local_sen_embed, doc_mask , dropout_rate )

    doc_mask_np = doc_mask.detach().cpu().numpy()
    remaining_mask_np = np.ones_like( doc_mask_np ).astype( bool ) | doc_mask_np
    extraction_mask_np = np.zeros_like( doc_mask_np ).astype( bool ) | doc_mask_np

    sen_logits_list = []
    sen_indices_list = []
    stop_logits_list = []
    stop_loss_mask_list = []

    extraction_context_embed = None
    for step in range( min(valid_sen_idxs.shape[1], max_extracted_sentences_per_document ) ):
        remaining_mask = torch.from_numpy( remaining_mask_np ).to(device)
        extraction_mask = torch.from_numpy( extraction_mask_np ).to(device)
        if step > 0:
            extraction_context_embed = extraction_context_decoder( local_sen_embed, remaining_mask, extraction_mask, dropout_rate )
        sen_logits, stop_logits = extractor( local_sen_embed, global_context_embed, extraction_context_embed, extraction_mask , dropout_rate )
        sen_logits = sen_logits.masked_fill( extraction_mask, -1e9 )
        sen_indices = valid_sen_idxs[:, step]

        sen_logits_list.append( sen_logits.unsqueeze(1) )
        sen_indices_list.append( sen_indices.unsqueeze(1) )
        stop_logits_list.append( stop_logits.unsqueeze(1) )
        stop_loss_mask_list.append( stop_loss_mask[:, step:step+1] )

        for doc_i in range( num_documents ):
            sen_i = sen_indices[ doc_i ].item()
            if sen_i != -1:
                remaining_mask_np[doc_i,sen_i] = False
                extraction_mask_np[doc_i,sen_i] = True

    sen_logits_list = torch.cat(sen_logits_list, dim = 1 ).view(-1, num_sentences)
    sen_indices_list = torch.cat(sen_indices_list, dim = 1 ).view(-1)
    stop_logits_list = torch.cat(stop_logits_list, dim = 1 ).view(-1)
    stop_loss_mask_list = torch.cat( stop_loss_mask_list, dim = 1 ).view(-1)

    ce_loss = ce_loss_criterion( sen_logits_list, sen_indices_list )
    ce_loss = ce_loss.sum() / ( (sen_indices_list != -1).sum() + 1e-9  )

    stop_loss = bce_loss_criterion( stop_logits_list, (sen_indices_list == -1).to(torch.float32)  )
    stop_loss = (stop_loss*stop_loss_mask_list).sum() / ( stop_loss_mask_list.sum() + 1e-9  )

    loss = ce_loss + stop_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

def calculate_exact_match_ratio(true_labels, predicted_labels):
    exact_match = 0
    for i in range(len(true_labels)):
        if true_labels[i] == predicted_labels[i]:
            exact_match += 1
    return exact_match / len(true_labels)

@torch.no_grad()
def validate(debug = False):
    predicted_indices = []
    label_indices = []

    predicted_sentences = []
    label_sentences = []

    for batch in val_data_loader:
        seqs, valid_sen_idxs, doc_mask, stop_loss_mask = [_.to(device) for _ in batch]

        num_sentences = seqs.size(1)
        local_sen_embed  = local_sentence_encoder_ema( seqs.view(-1, seqs.size(2) )  )
        local_sen_embed = local_sen_embed.view( -1, num_sentences, local_sen_embed.size(1) )
        global_context_embed = global_context_encoder_ema( local_sen_embed, doc_mask  )

        num_documents = seqs.size(0)
        doc_mask = doc_mask.detach().cpu().numpy()
        remaining_mask_np = np.ones_like( doc_mask ).astype( bool ) | doc_mask
        extraction_mask_np = np.zeros_like( doc_mask ).astype( bool ) | doc_mask

        done_list = []
        extraction_context_embed = None

        for step in range(max_extracted_sentences_per_document):
            remaining_mask = torch.from_numpy( remaining_mask_np ).to(device)
            extraction_mask = torch.from_numpy( extraction_mask_np ).to(device)
            if step > 0:
                extraction_context_embed = extraction_context_decoder_ema( local_sen_embed, remaining_mask, extraction_mask )
            sen_logits, stop_logits = extractor_ema( local_sen_embed, global_context_embed, extraction_context_embed , extraction_mask  )
            sen_logits = sen_logits.masked_fill( extraction_mask, -1e9 )
            p_stop = stop_logits.sigmoid()
            stop_action = p_stop > p_stop_thres

            done = stop_action | torch.all(extraction_mask, dim = 1)
            if len(done_list) > 0:
                done = torch.logical_or(done_list[-1], done)
            if torch.all( done ):
                break

            sen_indices = torch.argmax(sen_logits, dim =1)
            done_list.append(done)

            for doc_i in range( num_documents ):
                if not done[doc_i]:
                    sen_i = sen_indices[ doc_i ].item()
                    remaining_mask_np[doc_i,sen_i] = False
                    extraction_mask_np[doc_i,sen_i] = True

        predicted_indices += [ set(np.argwhere( remaining_mask_np[doc_i] == False )[:,0].tolist()) for doc_i in range( remaining_mask_np.shape[0] ) ]
        label_indices += [ set(valid_sen_idxs[doc_i][valid_sen_idxs[doc_i]!=-1].detach().cpu().numpy()) for doc_i in range( valid_sen_idxs.size(0) ) ]

        predicted_sentences += [ [ vocab.seq2sent( seqs[doc_i][sen_idx].cpu().numpy() ) for sen_idx in np.argwhere( remaining_mask_np[doc_i] == False )[:,0]] for doc_i in range( remaining_mask_np.shape[0] )   ]
        label_sentences += [ [ vocab.seq2sent( seqs[doc_i][sen_idx].cpu().numpy() ) for sen_idx in valid_sen_idxs[doc_i][valid_sen_idxs[doc_i]!=-1].detach().cpu().numpy()] for doc_i in range( valid_sen_idxs.size(0) )   ]

    score = calculate_exact_match_ratio( label_indices, predicted_indices )
    if not debug:
        return score
    else:
        return score, label_indices, predicted_indices, predicted_sentences, label_sentences

/home/scieditor/anaconda3/envs/pdf_vqa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|█████████████████████████████████████████████████| 4/4 [00:00<00:00,  9.24it/s]


## Start Training
The overall training process takes around **5 hours** using a single P100 GPU (available on Kaggle).

On Google Colab with a T4 GPU, it will take longer, roughly **9 hours** (Not recommended since Colab might automatically lose connection)

In [9]:
seed = 2023
data_folder = "pdfvqa/"
training_doc_info_path = data_folder + "/train_doc_info.pkl"
training_dataframe_path = data_folder + "/train_dataframe.csv"
validation_doc_info_path = data_folder + "/val_doc_info.pkl"
validation_dataframe_path = data_folder + "/val_dataframe.csv"
test_doc_info_path = data_folder + "/test_doc_info.pkl"
test_dataframe_path = data_folder + "/test_dataframe_without_answer.csv"
model_folder = "model/memsum_dqa/"
log_folder = "log/memsum_dqa/"
vocabulary_file_name = "model/word_embedding/vocabulary_200dim.pkl"
pretrained_unigram_embeddings_file_name = "model/word_embedding/unigram_embeddings_200dim.pkl"
num_heads = 8
hidden_dim = 1024
num_ehe_layers = 3
max_seq_len = 200
max_doc_len = 200
num_of_epochs = 50
print_every = 100
save_every = 200
validate_every = 200
restore_old_checkpoint = 1
learning_rate = 1e-4
warmup_step = 500
weight_decay = 0
dropout_rate = 0.2
n_device = 1
batch_size_per_device = 8
max_extracted_sentences_per_document = 4
moving_average_decay = 0.99
p_stop_thres = 0.2

set_seed(seed)

os.makedirs(log_folder, exist_ok=True)
os.makedirs(model_folder, exist_ok=True)

train_log_out_file = log_folder + "/train.log"
val_log_out_file = log_folder + "/val.log"

training_corpus = load_corpus( training_doc_info_path, training_dataframe_path  )
validation_corpus = load_corpus( validation_doc_info_path, validation_dataframe_path )

with open( vocabulary_file_name, "rb") as f:
    words = pickle.load(f)
with open(pretrained_unigram_embeddings_file_name, "rb") as f:
    pretrained_embedding = pickle.load(f)
vocab = Vocab(words)
vocab_size, embed_dim = pretrained_embedding.shape

train_dataset = ExtractionDataset(  training_corpus,  vocab , max_seq_len,  max_doc_len)
train_data_loader = DataLoader( train_dataset, batch_size=batch_size_per_device * n_device , shuffle=True, num_workers=4, worker_init_fn = lambda x:[np.random.seed( seed+x ), torch.manual_seed( seed + x) ], drop_last= True,  pin_memory= True )
total_number_of_samples = train_dataset.__len__()
val_dataset = ExtractionDataset( validation_corpus, vocab, max_seq_len, max_doc_len )
val_data_loader = DataLoader( val_dataset, batch_size=batch_size_per_device * n_device , shuffle=False, num_workers=4, worker_init_fn = lambda x:[np.random.seed( seed+x ), torch.manual_seed( seed + x) ], drop_last= False,  pin_memory= True)

local_sentence_encoder = LocalSentenceEncoder( vocab_size,vocab.pad_index, embed_dim,num_heads,hidden_dim,pretrained_embedding )
global_context_encoder = GlobalContextEncoder( embed_dim, num_heads, hidden_dim )
extraction_context_decoder = ExtractionContextDecoder( embed_dim, num_heads, hidden_dim, num_ehe_layers )
extractor = Extractor( embed_dim, num_heads )

# restore most recent checkpoint
if restore_old_checkpoint:
    ckpt = load_model( model_folder )
else:
    ckpt = None

if ckpt is not None:
    local_sentence_encoder.load_state_dict( ckpt["local_sentence_encoder"] )
    global_context_encoder.load_state_dict( ckpt["global_context_encoder"] )
    extraction_context_decoder.load_state_dict( ckpt["extraction_context_decoder"] )
    extractor.load_state_dict( ckpt["extractor"] )
    LOG("model restored!", train_log_out_file)
    print("model restored!")

gpu_list = np.arange(n_device).tolist()
device = torch.device(  "cuda:%d"%( gpu_list[0] ) if torch.cuda.is_available() else "cpu" )

local_sentence_encoder_ema = copy.deepcopy( local_sentence_encoder ).to(device)
global_context_encoder_ema = copy.deepcopy( global_context_encoder  ).to(device)
extraction_context_decoder_ema = copy.deepcopy( extraction_context_decoder ).to(device)
extractor_ema = copy.deepcopy( extractor ).to(device)

local_sentence_encoder.to(device)
global_context_encoder.to(device)
extraction_context_decoder.to(device)
extractor.to(device)

if device.type == "cuda" and n_device > 1:
    local_sentence_encoder = nn.DataParallel( local_sentence_encoder, gpu_list )
    global_context_encoder = nn.DataParallel( global_context_encoder, gpu_list )
    extraction_context_decoder = nn.DataParallel( extraction_context_decoder, gpu_list )
    extractor = nn.DataParallel( extractor, gpu_list )

    local_sentence_encoder_ema = nn.DataParallel( local_sentence_encoder_ema, gpu_list )
    global_context_encoder_ema = nn.DataParallel( global_context_encoder_ema, gpu_list )
    extraction_context_decoder_ema = nn.DataParallel( extraction_context_decoder_ema, gpu_list )
    extractor_ema = nn.DataParallel( extractor_ema, gpu_list )

    model_parameters = [ par for par in local_sentence_encoder.module.parameters() if par.requires_grad  ]  + \
                    [ par for par in global_context_encoder.module.parameters() if par.requires_grad  ]   + \
                    [ par for par in extraction_context_decoder.module.parameters() if par.requires_grad  ]  + \
                    [ par for par in extractor.module.parameters() if par.requires_grad  ]
else:
    model_parameters =  [ par for par in local_sentence_encoder.parameters() if par.requires_grad  ]  + \
                    [ par for par in global_context_encoder.parameters() if par.requires_grad  ]   + \
                    [ par for par in extraction_context_decoder.parameters() if par.requires_grad  ]  + \
                    [ par for par in extractor.parameters() if par.requires_grad  ]

optimizer = torch.optim.Adam( model_parameters , lr= learning_rate , weight_decay = weight_decay )

if ckpt is not None:
    try:
        optimizer.load_state_dict( ckpt["optimizer"] )
        LOG("optimizer restored!", train_log_out_file)
        print("optimizer restored!")
    except:
        pass

current_epoch = 0
current_batch = 0

if ckpt is not None:
    current_batch = ckpt["current_batch"]
    current_epoch = int( current_batch * batch_size_per_device * n_device / total_number_of_samples )
    LOG("current_batch restored!", train_log_out_file)
    print("current_batch restored!")

ce_loss_criterion = nn.CrossEntropyLoss( reduction="none", ignore_index=-1 )
bce_loss_criterion = nn.BCEWithLogitsLoss( reduction="none" )

max_val_score = None
best_ckpt_step = None

for epoch in range( current_epoch, num_of_epochs ):
    running_loss = 0

    for count, batch in tqdm(enumerate(train_data_loader)):
        loss = train_iteration(batch)
        running_loss += loss

        update_moving_average(  local_sentence_encoder_ema,  local_sentence_encoder, moving_average_decay)
        update_moving_average(  global_context_encoder_ema ,  global_context_encoder, moving_average_decay)
        update_moving_average(  extraction_context_decoder_ema,  extraction_context_decoder, moving_average_decay)
        update_moving_average(  extractor_ema,  extractor, moving_average_decay)

        current_batch +=1
        if current_batch % print_every == 0:
            current_learning_rate = get_lr( optimizer )[0]
            LOG( "[current_batch: %05d] loss: %.3f, learning rate: %f"%( current_batch, running_loss/print_every,  current_learning_rate  ), train_log_out_file )
            print( "[current_batch: %05d] loss: %.3f, learning rate: %f"%( current_batch, running_loss/print_every, current_learning_rate  ) )
            os.system( "nvidia-smi > %s/gpu_usage.log"%( log_folder ) )
            running_loss = 0

        if ( validate_every != 0 and  current_batch % validate_every == 0 ) or count == len(train_data_loader) - 1:
            print("Starting validation ...")
            LOG("Starting validation ...", train_log_out_file)
            # validation
            val_score = validate()

            ## only save the checkpoint that performs best on validation set so far
            if max_val_score is None or val_score > max_val_score:
                max_val_score = val_score
                best_ckpt_step = current_batch
                save_model(  {
                    "current_batch":current_batch,
                    "local_sentence_encoder": local_sentence_encoder_ema,
                    "global_context_encoder": global_context_encoder_ema,
                    "extraction_context_decoder":extraction_context_decoder_ema,
                    "extractor":extractor_ema,
                    "optimizer": optimizer.state_dict()
                    } , model_folder+"/model_batch_%d.pt"%(current_batch), max_to_keep = 1 )

            if max_val_score is not None:
                print("[current_batch: %05d] val: %.4f, [best_batch: %05d] best val: %.4f"%(current_batch, val_score, best_ckpt_step, max_val_score))
                LOG("[current_batch: %05d] val: %.4f, [best_batch: %05d] best val: %.4f"%(current_batch, val_score, best_ckpt_step, max_val_score), val_log_out_file)
            else:
                print("[current_batch: %05d] val: %.4f"%(current_batch, val_score ))
                LOG("[current_batch: %05d] val: %.4f"%(current_batch, val_score ), val_log_out_file)


99it [00:26,  3.84it/s]

[current_batch: 00100] loss: 4.110, learning rate: 0.000100


199it [00:52,  3.79it/s]

[current_batch: 00200] loss: 3.368, learning rate: 0.000100
Starting validation ...


200it [00:59,  2.30s/it]

...

[current_batch: 24600] val: 0.3546, [best_batch: 11000] best val: 0.4131


492it [02:30,  3.84it/s]

Starting validation ...


493it [02:37,  3.14it/s]

[current_batch: 24650] val: 0.3528, [best_batch: 11000] best val: 0.4131


# Load the checkpoint that performs best on the validation set

In [10]:
model_ckpt_path = sorted(glob("model/memsum_dqa/*.pt"), key = os.path.getmtime  )[-1]
print("Final model checkpoint path:", model_ckpt_path)

Final model checkpoint path: model/memsum_dqa/model_batch_11000.pt


In [11]:
memsum_pdf_vqa = MemSumPDFVQA( model_ckpt_path,
                               "model/word_embedding/vocabulary_200dim.pkl",
                               0
                             )

# Validate

In [12]:
val_corpus, paper_dict = load_dataset_for_inference( validation_doc_info_path, validation_dataframe_path )
for example in val_corpus:
    prediction = memsum_pdf_vqa.inference(
                                            example["question"],
                                            example["question_type"],
                                            paper_dict[ example["pmcid"] ],
                                            p_stop_thres = 0.2,
                                            max_extracted_sentences_per_document = 4
                                         )
    example["prediction_global_ids"] = set(prediction)

In [13]:
#### question type "parent_relationship_understanding"
label_ids, pred_ids = [], []
for example in val_corpus:
    if example["question_type"] == "parent_relationship_understanding":
        label_ids.append( example["answer_global_ids"] )
        pred_ids.append( example["prediction_global_ids"] )
print( "parent_relationship_understanding:", calculate_exact_match_ratio( label_ids, pred_ids  ) )

#### question type "child_relationship_understanding"
label_ids, pred_ids = [], []
for example in val_corpus:
    if example["question_type"] == "child_relationship_understanding":
        label_ids.append( example["answer_global_ids"] )
        pred_ids.append( example["prediction_global_ids"] )
print( "child_relationship_understanding:", calculate_exact_match_ratio( label_ids, pred_ids  ) )

#### question type overall
label_ids, pred_ids = [], []
for example in val_corpus:
    label_ids.append( example["answer_global_ids"] )
    pred_ids.append( example["prediction_global_ids"] )
print( "overall:", calculate_exact_match_ratio( label_ids, pred_ids  ) )

parent_relationship_understanding: 0.33905579399141633
child_relationship_understanding: 0.7130434782608696
overall: 0.41308089500860584


# Test

In [14]:
corpus, paper_dict = load_dataset_for_inference( test_doc_info_path, test_dataframe_path )
predicted_labels = []
for idx in range(len(corpus)):
    question = corpus[idx]["question"]
    question_type = corpus[idx]["question_type"]
    doc_content = paper_dict[ corpus[idx]["pmcid"] ]
    predicted_labels.append(  memsum_pdf_vqa.inference( question, question_type, doc_content,
                                                            p_stop_thres = 0.2, max_extracted_sentences_per_document = 4 ) )

id_list = range(0, len(predicted_labels))
df = pd.DataFrame(id_list, columns=['id'])
df['answer'] = predicted_labels

df.to_csv('submission.csv', index=False)